In [ ]:
#import of the libraries

import streamlit as st
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

#viz 
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import matplotlib.image as mpimg

#scraping
from linkpreview import link_preview

#viz wordcloud
from wordcloud import WordCloud


#title of the page
st.title('Balance ton film')

#import of the datasets for the two recommender
df = pd.read_csv('file:///C:/Users/jo_18/Desktop/FormationDATA/Projet_recommandation/links_movies.csv')
df2 = pd.read_csv('file:///C:/Users/jo_18/Desktop/FormationDATA/Projet_recommandation/userId_movieId_rating.csv',index_col=0)

#modification of df
df['imdbId'] = df['imdbId'].astype(str)
df['imdbId'] = df['imdbId'].apply(lambda x: x.zfill(7))
df = df.drop(columns = 'Unnamed: 0')


#logo
image = Image.open('large_moviewatchers.png')

st.sidebar.image(image,
        use_column_width=True)



#wordcloud function

#import of the tags
tags_movies = pd.read_csv('file:///C:/Users/jo_18/Desktop/FormationDATA/Projet_recommandation/tags_movies.csv')

def wordcloud(movie):
    if movie not in tags_movies['title'].tolist():
        nowords = print('No tags found') #we notice that 200 movies / 9742 do not have tags
        return nowords
    else:    
        movie_Id = int(tags_movies[tags_movies['title'] == movie]['movieId'].unique())
        liste_tags= list(tags_movies[tags_movies['movieId'] == movie_Id]['tag'])
        
        text = str(liste_tags)
        text = text.replace("'","")
        
        fig, ax = plt.subplots(1,1,figsize = (4,2))
        ax1 = plt.subplot(111)
        
        wordcloud = WordCloud(width = 800, height= 400, background_color = 'white',max_words = 40,colormap = 'inferno').generate(text)
             
        ax1.imshow(wordcloud, interpolation="bilinear")
        ax1.axis("off")
        ax1.margins(x=0, y=0)
        
        return st.pyplot(fig)
    
    
#function recommender 1 with neirest neighbours
def recommand_5(movie):

    neigh = NearestNeighbors(n_neighbors=6) #initialization of the model
    
    neigh.fit(df.iloc[:,4:24]) #we created dummies for each genre in df
    
    input_movie = np.array(df[df['title']==movie])[:,4:24] 
    
    #we calculate the distances between each movie
    neigh.kneighbors(input_movie) 
    
    #we select the 5 closiest movies
    index_closest = list(neigh.kneighbors(input_movie)[1][0,0:]) 
    if df[df['title'] == movie].index.isin(index_closest):
        index_closest.remove(df[df['title'] == movie].index) 
    else:
        index_closest = list(neigh.kneighbors(input_movie)[1][0,:5]) 
    liste_recommandations = list(df[df.index.isin(index_closest)].iloc[:,1])
    return liste_recommandations



#function recommender 2 with ratings correlation
def corr_rate5(movie):
    movieid=str(df[df['title']== movie]['movieId'].tolist()[0])
    
    data={} 
    _,c=df2.shape

    for n in range(c):
        data[df2.columns[n]]=df[df['movieId']==int(df2.columns[n])]['title'].unique().tolist()[0],df2.iloc[:,n].corr(df2[movieid]),df2.iloc[:,n][df2.iloc[:,n]!=0].count()
        
    resultat=pd.DataFrame.from_dict(data, orient='index',columns=['title', 'corr', 'nb_rating'])

    resultat.drop([movieid],inplace=True)

    return resultat.sort_values(by='corr',ascending=False)['title'].head().tolist()


#functiun input
def input_sys_1():
    
    movie = st.selectbox(
    'Veuillez sélectionner un film que vous appréciez :', #EN: 'Please select a movie that you love watching:'
     df['title'])
    
    return movie



#function to catch url
def catch_url(liste_movie):
    liste_url = []
    for i in liste_movie:
        imdbId = df[df['title'] == i]['imdbId']
        imdbId = imdbId.iloc[0]
        url_imbd = 'http://www.imdb.com/title/tt'+ imdbId +'/'
        liste_url.append(url_imbd)
    return liste_url


#function to scrap and display the posters of each movie of the 5 recommended (with a matplotlib figure)

def get_display_image(liste_url):
    
    liste_image = []
    for url in liste_url:
        preview = link_preview(url)
        link_image = preview.image 
        response = requests.get(link_image) 
        image = Image.open(BytesIO(response.content)) 
        image = image.resize((170, 290), Image.ANTIALIAS) 
        liste_image.append(image)
        
    #figure
    fig, ax = plt.subplots(1,5,figsize = (25,10))
    for i in range(len(liste_image)):
        imagebox = OffsetImage(liste_image[i], zoom=1.2) 
        ab = AnnotationBbox(imagebox, (0.4, 0.6))  
        ax[i].add_artist(ab)
        ax[i].axis('off')
        
        
    return st.pyplot(fig) 


#global function 1
def recommandation_sys_1(movie):
    liste_movie = recommand_5(movie)
    liste_url = catch_url(liste_movie)
    liste_affiches = get_display_image(liste_url)
    return liste_affiches

#global function 2
def recommandation_sys_2(movie):
    liste_movie = corr_rate5(movie)
    liste_url = catch_url(liste_movie)
    liste_affiches = get_display_image(liste_url)
    return liste_affiches

#run

#input
movie = input_sys_1()

#wordcloud
st.header("Comment les utilisateurs définissent-ils ce film ?") #EN: 'How users define the movie:'

wordcloud(movie)

#subheader 1
st.header('Dans le même genre...')

#recommender 1
recommandation_sys_1(movie)


#subheader 2
st.header('Les utilisateurs qui ont aimé ce film, ont aussi aimé...')

#gif
file_ = open("giphy1.gif", "rb")
contents = file_.read()
data_url = base64.b64encode(contents).decode("utf-8")
file_.close()

st.markdown(
    f'<img src="data:image/gif;base64,{data_url}" alt="cat gif">',
    unsafe_allow_html=True,
)


#recommender 2
recommandation_sys_2(movie)


#subheader 3
st.header('Quelques statistiques sur la base de données...')


#viz
image2 = Image.open('viz.png')

st.image(image2,
        use_column_width=True)